# Pandas ile Veri Biçimlendirme (Data Formatting) Rehberi

Bu rehber, Pandas kullanarak verilerinizi analiz veya sunum için nasıl biçimlendireceğinizi ve dönüştüreceğinizi anlatır. Veri biçimlendirme; metinleri temizlemek, sayısal formatları ayarlamak, verileri gruplamak ve tabloları görsel olarak daha anlaşılır hale getirmek gibi çeşitli işlemleri kapsar.

## 1. Kurulum ve Örnek Veri Seti

Biçimlendirme ihtiyacı olan, çeşitli ve "kirli" veri türleri içeren bir örnek DataFrame oluşturalım.

In [ ]:
import pandas as pd
import numpy as np

data = {
    'urun_adi': ['  Elma  ', 'ARMUT', 'çilek', 'muzlu kek', ' Siyah Üzüm '],
    'fiyat': ['$1.99', '€2,50', '3.00 TL', '$0.99', '€4,20'],
    'stok_miktari': [150, 200, 120, 80, 100],
    'kategori': ['Meyve', 'Meyve', 'Meyve', 'İşlenmiş Gıda', 'Meyve'],
    'son_satis_tarihi': ['15-08-2023', '16.08.2023', '2023/08/17', '18-08-2023', '19.08.2023'],
    'musteri_notu': ['Çok taze', 'Biraz ezik geldi', 'Tadı güzeldi', 'Çocuklar sevdi', 'Rengi harika']
}

df = pd.DataFrame(data)

print("Orjinal, Biçimlendirilmemiş Veri Seti:")
display(df)

## 2. Metin (String) Verilerini Biçimlendirme

Metin verileri genellikle tutarsızdır. `.str` erişimcisi (accessor) ile metinler üzerinde güçlü işlemler yapabiliriz.

In [ ]:
# Harf Büyüklüğünü Standartlaştırma
df['urun_adi_temiz'] = df['urun_adi'].str.title() # Her Kelimenin İlk Harfi Büyük

# Baştaki ve Sondaki Boşlukları Temizleme
df['urun_adi_temiz'] = df['urun_adi_temiz'].str.strip()

# Metin İçeriğini Değiştirme
df['fiyat_temiz'] = df['fiyat'].str.replace(',', '.').str.replace(' TL', '')

print("Metin temizliği sonrası ilk sonuçlar:")
display(df[['urun_adi_temiz', 'fiyat_temiz']])

## 3. Sayısal Verileri Biçimlendirme ve Dönüştürme

Metin olarak saklanan sayıları doğru veri tipine çevirmek, matematiksel işlemler yapabilmek için zorunludur.

In [ ]:
# Para birimi simgelerini kaldırıp sayısal tipe çevirme
df['fiyat_sayisal'] = df['fiyat_temiz'].str.extract(r'(\d+\.\d+)').astype(float)

print("Temizlenmiş ve sayısal tipe dönüştürülmüş fiyatlar:")
display(df[['urun_adi_temiz', 'fiyat_sayisal', 'fiyat_sayisal']])
print("\nYeni sütunun veri tipi:", df['fiyat_sayisal'].dtype)


### Sürekli Değişkenleri Kategorik Hale Getirme (Binning)

Bazen `stok_miktari` gibi sürekli bir değişkeni, 'Az', 'Orta', 'Çok' gibi kategorilere ayırmak analizleri kolaylaştırır. `pd.cut` bu iş için idealdir.

In [ ]:
# Stok miktarını kategorilere ayıralım
bins = [0, 100, 180, np.inf] # Sınır değerleri
labels = ['Az', 'Orta', 'Çok']   # Kategorilerin etiketleri

df['stok_kategorisi'] = pd.cut(df['stok_miktari'], bins=bins, labels=labels, right=False)

print("Stok miktarına göre oluşturulan kategoriler:")
display(df[['stok_miktari', 'stok_kategorisi']])

## 4. Tarih/Zaman Verilerini Biçimlendirme

Farklı formatlardaki tarihleri standart bir `datetime` nesnesine çevirmek, tarih bazlı analizler için kritiktir.

In [ ]:
# Tarih sütununu standart datetime formatına çevirme
df['son_satis_tarihi_dt'] = pd.to_datetime(df['son_satis_tarihi'], format='mixed', dayfirst=True)

# Tarih bileşenlerini yeni sütunlara çıkarma
df['satis_ayi'] = df['son_satis_tarihi_dt'].dt.month_name(locale='tr_TR.utf8')
df['satis_gunu'] = df['son_satis_tarihi_dt'].dt.day_name(locale='tr_TR.utf8')

print("Biçimlendirilmiş tarih verileri:")
display(df[['son_satis_tarihi_dt', 'satis_ayi', 'satis_gunu']])

## 5. Görüntüleme (Display) Biçimlendirmesi - `Styler` Sınıfı

Bu yöntemler, altta yatan veriyi **değiştirmez**, sadece Jupyter Notebook gibi ortamlarda tablonun **görünümünü** iyileştirir. Raporlama ve sunum için harikadır.

In [1]:
# Sunum için basit bir DataFrame oluşturalım
df_sunum = df[['urun_adi_temiz', 'fiyat_sayisal', 'stok_miktari']].copy()
df_sunum.rename(columns={'urun_adi_temiz': 'Ürün Adı', 'fiyat_sayisal': 'Fiyat', 'stok_miktari': 'Stok'}, inplace=True)
df_sunum['karlilik_orani'] = np.random.rand(len(df_sunum)) - 0.3 # Rastgele karlılık oranı

# Styler nesnesi ile biçimlendirme
df_sunum.style
 .format({
     'Fiyat': '₺{:.2f}', # Para birimi formatı
     'karlilik_orani': '{:.2%}' # Yüzde formatı
 })
 .background_gradient(cmap='viridis', subset=['Stok']) # Stok için ısı haritası
 .bar(subset=['Fiyat'], color='#5fba7d') # Fiyat için veri çubukları
 .applymap(lambda x: 'color: red' if x < 0 else 'color: black', subset=['karlilik_orani']) # Negatif karlılığı kırmızı yap
 .set_caption('Ürün Satış Raporu') # Tablo başlığı


IndentationError: unexpected indent (1634180578.py, line 8)

## Sonuç

Veri biçimlendirme, ham veriyi anlamlı ve işlenebilir bir hale getirmenin temel adımıdır. Pandas, bu süreç için metin manipülasyonundan (`.str`), sayısal dönüşümlere (`.astype`, `pd.cut`) ve sunum kalitesinde görsel biçimlendirmeye (`.style`) kadar geniş bir araç yelpazesi sunar.